<!--NAVIGATION-->
< [ANOVA](15-anova.ipynb) | [Main Contents](Index.ipynb) | [Multiple Variables and Interactions](17-MulExplInter.ipynb)>

# Linear Models: Multiple explanatory variables <span class="tocSkip">

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span><ul class="toc-item"><li><span><a href="#Loading-the-data" data-toc-modified-id="Loading-the-data-1.1">Loading the data</a></span></li><li><span><a href="#Boxplots-within-groups" data-toc-modified-id="Boxplots-within-groups-1.2">Boxplots within groups</a></span></li><li><span><a href="#lattice-again" data-toc-modified-id="lattice-again-1.3"><code>lattice</code> again</a></span></li><li><span><a href="#Barplots-again" data-toc-modified-id="Barplots-again-1.4">Barplots again</a></span></li><li><span><a href="#Plotting-means-and-confidence-intervals" data-toc-modified-id="Plotting-means-and-confidence-intervals-1.5">Plotting means and confidence intervals</a></span></li><li><span><a href="#Multiple-explanatory-variables" data-toc-modified-id="Multiple-explanatory-variables-1.6">Multiple explanatory variables</a></span></li><li><span><a href="#Predicted-values" data-toc-modified-id="Predicted-values-1.7">Predicted values</a></span></li></ul></li></ul></div>

# Introduction

Aims of this chapter[$^{[1]}$](#fn1):

Including several explanatory variables in a model

Interpreting summary tables for more complex models

## Loading the data


$\star$ Create a new blank script called `MulExpl.R` in your `Code` directory and add some introductory comments.

Use `load('../Data/mammals.Rdata')` to load the data saved at the end of [ANOVA](15-anova.ipynb). Look back at the end of the previous chapter to see how you saved the RData file. If `mammals.Rdata` is missing, just import the data again using `read.csv(“../Data/MammalData.csv”)` and add the `log C Value` column to the imported data frame again (got back to [ANOVA](15-anova.ipynb) and have a look if you have forgotten how).

Use `ls` and `str` to check that the data has loaded correctly.

The models we looked at in [ANOVA](15-anova.ipynb) explored whether the log genome size (C value, in picograms) of terrestrial mammals varied with trophic level and whether or not the species is ground dwelling. We will now look at a single model that includes both explanatory variables.

The first thing to do is look at the data again. In [Regress](14-regress.ipynb), we asked if carnivores or herbivores had larger genomes. Now we want to ask questions like: do ground-dwelling carnivores have larger genomes than arboreal or flying omnivores? We need to look at plots within groups.

Before we do that, there is a lot of missing data in the data frame and
we should make sure that we are using the same data for our plots and
models. We will subset the data down to the complete data for the three
variables:

In [ ]:
> mammals <- subset(mammals, select = c(GroundDwelling, TrophicLevel, 
logCvalue))
> mammals <- na.omit(mammals)
> str(mammals)

'data.frame':   259 obs. of  3 variables:
 $GroundDwelling: Factor w/ 2 levels "No","Yes": 2 2 2 2 2 1 2 1 1 1 ...
 $TrophicLevel  : Factor w/ 3 levels "Carnivore","Herbivore",..: 1 2 2 2 3 3 3 2 2 3 ...
 $logCvalue     : num  0.94 1.322 1.381 1.545 0.888 ...
  - attr(*, "na.action")=Class 'omit'  Named int [1:120] 2 4 7 9 10 11 14 15 20 21 ...
   .. ..- attr(*, "names")= chr [1:120] "2" "4" "7" "9" ...

## Boxplots within groups

In Chapter \[ch:regress\], we used the `subset` option to fit
a model just to dragonflies. You can use `subset` with plots
too.

$\star$ Add `par(mfrow=c(1,2))` to your script to split the graphics
into two panels.

Copy the code from [ANOVA](15-anova.ipynb) to create a boxplot of genome
size by trophic level into your script.

Using this, and adding a `subset` option to the code,
generate the plots shown in the figure below.

You can use the option `main` to add titles to a plot.

<a id="fig:boxplots"></a>
<figure>
    <img src="./graphics/boxplots.svg" alt="boxplots" style="width:100%">
    <small> 
        <center>
            <figcaption> 
           Figure 1
            </figcaption>
        </center>
    </small>
</figure>

## `lattice` again

Recall that the <span>lattice</span> package provides some very neat
extra ways to plot data in groups. They look pretty but the downside is
that they don't use the same graphics system — all those
<span>par</span> commands are useless for these graphs. The defaults
look good though!

In [ ]:
> library(lattice)
> bwplot(logCvalue ~ TrophicLevel | GroundDwelling, data= mammals)

<a id="fig:bwplots"></a>
<figure>
    <img src="./graphics/bwplots.svg" alt="bwplots" style="width:100%">
    <small> 
        <center>
            <figcaption> 
           Figure 2
            </figcaption>
        </center>
    </small>
</figure>

The code `logCvalue ~ TrophicLevel | GroundDwelling` means
plot the relationship between genome size and trophic level, but group
within levels of ground dwelling. We are using the function
`bwplot`, which is provided by `lattice` to create
box and whisker plots.

$\star$ Create the lattice plots above from within your script.

Rearrange this code to have three plots, showing the box and whisker
plots for `GroundDwelling`, grouped within the levels of
`TrophicLevel`.

Try reshaping the R plot window and running the command again. Lattice
tries to make good use of the available space when creating lattice
plots.

## Barplots again

We're going to make the barplot code from [Regress](14-regress.ipynb) even
more complicated! This time we want to know the mean log genome size
within combinations of `TrophicLevel` and
`GroundDwelling`. We can still use `tapply`,
providing more than one grouping factor. We create a set of grouping
factors like this:

In [ ]:
> groups <- list(mammals$GroundDwelling, mammals$TrophicLevel)
> groupMeans <- tapply(mammals$logCvalue, groups, FUN = mean)
> print(groupMeans)

     Carnivore Herbivore Omnivore
 No     0.9589     1.012    1.192
 Yes    1.2138     1.298    1.299

$\star$ Copy this code into your script and run it.

Use this code and the script from [ANOVA](15-anova.ipynb) to get the set of
standard errors for the groups `groupSE`. You should get
this:

In [ ]:
Carnivore Herbivore Omnivore
     No    0.04842   0.03419  0.02410
     Yes   0.05976   0.02787  0.03587

Now we can use <span>barplot</span>. The default option for a barplot of
a table is to create a stacked barplot, which is not what we want. The
option <span>beside=TRUE</span> makes the bars for each column appear
side by side.

Once again, we save the midpoints of the bars to add the
error bars. The other options in the code below change the colours of
the bars and the length of error bar caps.

In [ ]:
# get upper and lower standard error height
    > upperSE <- groupMeans + groupSE
    > lowerSE <- groupMeans - groupSE
# create barplot
    > barMids <- barplot(groupMeans, ylim=c(0, max(upperSE)), beside=TRUE, 
    ylab= ' log C value (pg) ' , col=c( ' white ' , ' grey70 '))
    > arrows(barMids, upperSE, barMids, lowerSE, ang=90, code=3, len=0.05)

<a id="fig:barplot_1"></a>
<figure>
    <img src="./graphics/barplot_1.svg" alt="barplot_1" style="width:100%">
    <small> 
        <center>
            <figcaption> 
           Figure 3
            </figcaption>
        </center>
    </small>
</figure>

$\star$ Generate the barplot above and then edit your script to change the
colours and error bar lengths to your taste.

## Plotting means and confidence intervals

We'll use the `plotmeans` function again as an exercise to
change graph settings and to prepare figures for write ups.

<a id="fig:plotmeans"></a>
<figure>
    <img src="./graphics/plotmeans.svg" alt="plotmeans" style="width:90%">
    <small> 
        <center>
            <figcaption> 
           Figure 4: Means and 95% confidence intervals for log genome size (picograms) in mammals for different trophic levels for a) ground dwelling species and b) other species.
            </figcaption>
        </center>
    </small>
</figure>

The default options in R use wide margins and spaced out axes and take
up a lot of space that could be used for plotting data. You've already
seen the `par` function and the options `mfrow`
for multiple plots and `mar` to adjust margin size. The
option `mgp` adjusts the placement of the axis label, tick
labels and tick locations. See `?par` for help on the these
options.

Adding large titles to graphs is also a bad idea — it uses lots of space
to explain something that should be in the figure legend. With multiple
plots in a figure, you have to label graphs so that the figure legend
can refer to them. You can add labels using
`text(x,y,'label')`.

A figure legend should give a clear stand-alone description of the whole
figure.

You *must* link from your text to your figures — a reader
has to know which figures refer to which results. So: ‘There are clear
differences in mean genome size between species at different trophic
levels and between ground dwelling and other species, [Figure 3](#fig:plotmeans)'.

$\star$ Use `plotmeans` from [ANOVA](15-anova.ipynb) and the `subset` option to generate the two plots below. You will need to
set the `ylim` option for the two plots to make them use the same $y$ axis.

Use `text` to add labels — the command
`par('usr')` will show you the limits of the plot
($x_{min}, x_{max}, y_{min}, y_{max}$) and help pick a location for the labels.

Change the `par` settings in your code and redraw the plots
to try and make better use of the space. In the example below, the box
shows the edges of the R graphics window.

## Multiple explanatory variables

All those graphs suggest:

Carnivores have smaller genome size; omnivores have larger genome size.

Herbivores are somewhere in between, but not consistently.

All ground dwelling mammals typically have larger genome sizes.

We suspected these things from [ANOVA](15-anova.ipynb), but now we can see
that they might have separate effects. We'll fit a linear model to
explore this and add the two explanatory variables together.

$\star$ This is an important section — read it through carefully and ask
questions if you are unsure. Copy the code into your script and add
comments. *Do not just jump to the next action item*!

In [ ]:
> model <- lm(logCvalue ~ TrophicLevel + GroundDwelling, data = mammals)    

We're going to do things right this time and check the model diagnostics
before we rush into interpretation.

In [ ]:
> par(mfrow=c(2,2))
> plot(model)

<a id="fig:plotmeans"></a>
<figure>
    <img src="./graphics/diagnostics1.svg" alt="diagnostics1">
    <small> 
        <center>
            <figcaption> 
           Figure 5
            </figcaption>
        </center>
    </small>
</figure>

There are six predicted values now - three trophic levels for each of
the two levels of ground dwelling. Those plots look ok so now we can
look at the analysis of variance table:

In [ ]:
> anova(model)

 Analysis of Variance Table
 
 Response: logCvalue
                 Df Sum Sq Mean Sq F value  Pr(>F)    
 TrophicLevel     2   0.81   0.407    7.86 0.00049 ***
 GroundDwelling   1   2.75   2.747   53.04 4.1e-12 ***
 Residuals      255  13.21   0.052                    
 ---
 Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

*Ignore the $p$ values*! Yes, they're highly significant
but we want to understand the model, not rubber stamp it with
‘significant'.

The sums of squares for the variables are both small compared to the
residual sums of squares — there is lots of unexplained variation. We
can calculate the $r^2$ as explained sums of squares over total sums of
squares:
$$\frac{0.81 + 2.75}{0.81 + 2.75 + 13.21} = \frac{3.56}{16.77} = 0.212$$

Trophic level explain much less variation than ground dwelling — this
makes intuitive sense from the plots since there are big differences
between [Figure 3](#fig:plotmeans)a/b, but small
differences within.

We could also calculate a significance for the whole model by merging
the terms. The total explained sums of squares of $0.81 + 2.75 
=3.56$ uses $2+1 =3$ degrees of freedom, so the mean sums of squares for
all the terms together is $3.56/3=1.187$. Dividing this by the residual
mean square of 0.052 gives an F of $1.187 / 0.052 = 22.83$.

Now we can look at the summary table to see the coefficients.

In [ ]:
summary(model) 

 Call:
 lm(formula = logCvalue ~ TrophicLevel + GroundDwelling, data = mammals)
 
 Residuals:
     Min      1Q  Median      3Q     Max 
 -0.4990 -0.1784 -0.0146  0.1250  0.8624 
 
 Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
 (Intercept)             0.9798     0.0354   27.68  < 2e-16 ***
 TrophicLevelHerbivore   0.0766     0.0397    1.93    0.055 .  
 TrophicLevelOmnivore    0.1727     0.0398    4.34  2.0e-05 ***
 GroundDwellingYes       0.2095     0.0288    7.28  4.1e-12 ***
 ---
 Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 
 
 Residual standard error: 0.228 on 255 degrees of freedom
 Multiple R-squared: 0.212, Adjusted R-squared: 0.203 
 F-statistic: 22.9 on 3 and 255 DF,  p-value: 3.6e-13 
 

Starting at the bottom, `summary` has again calculated $r^2$
for us and also an $F$ statistic for the whole model, which matches the
calculation above.

The other important bits are the four coefficients. The intercept is now
the reference level for two variables: it is the mean for carnivores
that are not ground dwelling. We then have differences from this value
for being an omnivore or herbivore and for being ground dwelling. There
is a big change in genome size associated with ground dwelling and
omnivory and both of these have large effects sizes, each introducing
about a 20% difference in genome size from the non-ground dwelling
carnivores. In contrast, herbivory makes a small difference — about 8%.
Because the difference is small and the standard error is large, the $t$
value suggests that this difference might arise just be chance. Put
another way, it isn't significant.

The table below shows how these four coefficients combine to give the
predicted values for each of the group means.

$$\begin{array}{|r|rrr|}
\hline
           & \textrm{Carnivore} & \textrm{Herbivore} & \textrm{Omnivore}\\
\hline
\textrm{Not ground} & {\it 0.98} = 0.98    & {\it 0.98  + 0.08} =1.06    & {\it 0.98 + 0.17} =1.15 \\
\textrm{Ground}     & {\it 0.98 + 0.21} = 1.19    & {\it 0.98  + 0.08   + 0.21} =1.27   & {\it 0.98 + 0.17  + 0.21} =1.36\\
\hline
\end{array}$$

Predicted values
----------------

Getting the model predictions by hand in this way is tedious and error
prone. There is handy function called `predict` which uses
the model directly to calculate values. The default is to give you the
prediction for each point in the original data, but you can also ask for
specific predictions.

The first thing to do is to set up a small data frame containing the
explanatory values we want to use. The variable names and the level name
have to match *exactly*, so we'll use the
`levels` function to get the names. We want to look at all
    six combinations, so we'll use the `rep` function to set this
up. The `each=2` option repeats each value twice in
succession; the `times=3` options repeats the whole set of
values three times.

In [ ]:
# data frame of combinations of variables
> gd <- rep(levels(mammals$GroundDwelling), times = 3)
> print(gd)

 [1] "No"  "Yes" "No"  "Yes" "No"  "Yes"
 
> tl <- rep(levels(mammals$TrophicLevel), each = 2)
> print(tl)

  [1] "Carnivore" "Carnivore" "Herbivore" "Herbivore" "Omnivore"  "Omnivore" 

> predVals <- data.frame(GroundDwelling = gd, TrophicLevel = tl)

Now we have the data frame of values we want, we can use `predict`. As
when we created log values, we can save the output back into a new
column in the data frame.

In [ ]:
> predVals$predict <- predict(model, newdata = predVals)
> print(predVals)

   GroundDwelling TrophicLevel predict
 1             No    Carnivore  0.9798
 2            Yes    Carnivore  1.1892
 3             No    Herbivore  1.0563
 4            Yes    Herbivore  1.2658
 5             No     Omnivore  1.1524
 6            Yes     Omnivore  1.3619

$\star$ These are in the same order as the bars from your barplot. Make a copy
of the barplot and arrows code and then add the code below to generate
the plot.

In [ ]:
> points(barMids, predVals$predict, col= ' red ' , pch=5)

<a id="fig:barplotpred"></a>
<figure>
    <img src="./graphics/barplotpred.svg" alt="barplotpred" style="width:60%">
    <small> 
        <center>
            <figcaption> 
           Figure 6
            </figcaption>
        </center>
    </small>
</figure>

The red points do not match to the calculated means. This is because the
model only includes a single difference between ground and non-ground
species, which has to be the same for each trophic group. That is, there
is no interaction between trophic level and ground / non-ground identity
of each species in the current model.

[MulExplInter](17-MulExplInter.ipynb) will look at interactions, which allows
these values to differ using an interaction term in the model.

---
<a id="fn1"></a>
[1]: Here you work with the script file `MulExpl.R`